# Table of Contents
 <p><div class="lev2 toc-item"><a href="#Illustrate-a-Graph-Flow-in-tf1" data-toc-modified-id="Illustrate-a-Graph-Flow-in-tf1-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Illustrate a Graph-Flow in tf1</a></div><div class="lev3 toc-item"><a href="#Linear-Regression" data-toc-modified-id="Linear-Regression-011"><span class="toc-item-num">0.1.1&nbsp;&nbsp;</span>Linear Regression</a></div>

In [6]:
# https://www.bookstack.cn/read/TensorFlow2.0/spilt.2.8a0a47774f201d90.md

In [2]:
# import tensorflow as tf
 
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()    

a = tf.placeholder(dtype=tf.int32)# 定义一个占位符Tensor
b = tf.placeholder(dtype=tf.int32)
c = a + b
 
a_ = input("a = ")# 从终端读入一个整数并放入变量a_
b_ = input("b = ")
 
sess = tf.Session()
c_ = sess.run(c, feed_dict={a: a_, b: b_})# feed_dict参数传入为了计算c所需要的张量的值
print("a + b = %d"% c_)

Instructions for updating:
non-resource variables are not supported in the long term
a = 1
b = 2
a + b = 3


2021-11-11 10:08:03.654727: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
a = tf.get_variable(name='a', shape=[])
initializer = tf.assign(a,0)# tf.assign(x, y)返回一个“将张量y的值赋给变量x”的操作
a_plus_1 = a +1# 等价于 a + tf.constant(1)
plus_one_op = tf.assign(a, a_plus_1)
 
sess = tf.Session()
sess.run(initializer)
for i in range(5):
    sess.run(plus_one_op)# 对变量a执行加一操作
    a_ = sess.run(a)# 获得变量a的值并存入a_
print(a_)

5.0


以下代码和上述代码等价，在声明变量时指定初始化器，并通过 tf.global_variables_initializer() 一次性初始化所有变量，在实际工程中更常用：

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()   

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
a = tf.get_variable(name='a', shape=[], initializer=tf.zeros_initializer)# 指定初始化器为全0初始化
a_plus_1 = a +1
plus_one_op = tf.assign(a, a_plus_1)
 
sess = tf.Session()
sess.run(tf.global_variables_initializer())# 初始化所有变量
for i in range(5):
    sess.run(plus_one_op)
    a_ = sess.run(a)
print(a_)

5.0


2021-11-11 10:11:57.009262: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Illustrate a Graph-Flow in tf1

### Linear Regression

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()  

Instructions for updating:
non-resource variables are not supported in the long term


与前面的NumPy和Eager Execution模式不同，TensorFlow的Graph Execution模式使用 符号式编程 来进行数值运算。首先，我们需要将待计算的过程抽象为数据流图，将输入、运算和输出都用符号化的节点来表达。然后，我们将数据不断地送入输入节点，让数据沿着数据流图进行计算和流动，最终到达我们需要的特定输出节点。以下代码展示了如何基于TensorFlow的符号式编程方法完成与前节相同的任务。其中， tf.placeholder() 即可以视为一种“符号化的输入节点”，使用 tf.get_variable() 定义模型的参数（Variable类型的张量可以使用 tf.assign() 进行赋值），而 sess.run(output_node, feed_dict={input_node: data}) 可以视作将数据送入输入节点，沿着数据流图计算并到达输出节点并返回值的过程。

In [2]:
import numpy as np

In [3]:
 
# 定义数据流图
learning_rate_ = tf.placeholder(dtype=tf.float32)
X_ = tf.placeholder(dtype=tf.float32, shape=[5])
y_ = tf.placeholder(dtype=tf.float32, shape=[5])
a = tf.get_variable('a', dtype=tf.float32, shape=[], initializer=tf.zeros_initializer)
b = tf.get_variable('b', dtype=tf.float32, shape=[], initializer=tf.zeros_initializer)


y_pred = a * X_ + b
loss = tf.constant(0.5) * tf.reduce_sum(tf.square(y_pred - y_))
 
# 反向传播，手动计算变量（模型参数）的梯度
# equivalent to 
#
# grad_a, grad_b = tf.gradients(loss, [a, b])
grad_a = tf.reduce_sum((y_pred - y_) * X_)
grad_b = tf.reduce_sum(y_pred - y_)
 
    
# 梯度下降法，手动更新参数
# equivalent to
# method-2:
# optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate_)
# grad = optimizer.compute_gradients(loss)
# train_op = optimizer.apply_gradients(grad)
#
# or method-3:
# train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate_).minimize(loss)

new_a = a - learning_rate_ * grad_a
new_b = b - learning_rate_ * grad_b
update_a = tf.assign(a, new_a)
update_b = tf.assign(b, new_b)
train_op = [update_a, update_b]






# 数据流图定义到此结束
# 注意，直到目前，我们都没有进行任何实质的数据计算，仅仅是定义了一个数据图
 

In [5]:
num_epoch = 10000
learning_rate = 1e-3

# Add Inputs
X = 5* np.random.random(size=(5, )) # generate 1000 random numbers between (0, 20)
y = 3 * X + np.random.normal(loc=0.0, scale=0.01, size=(5,))


with tf.Session() as sess:
    # 初始化变量a和b
    tf.global_variables_initializer().run()
    # 循环将数据送入上面建立的数据流图中进行计算和更新变量
    for e in range(num_epoch):
        sess.run(train_op, feed_dict={X_: X, y_: y, learning_rate_: learning_rate})
    print(sess.run([a, b]))

[2.9952414, 0.011332989]
